In [ ]:
import os
import cv2
import sys
import json
import math
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize
from skimage.io import imshow

from tqdm.notebook import tqdm as tqdm

In [ ]:
ori_img_path_list = glob.glob('./data/raw_data/*.PNG') 
f = open('./label_json/key_point_json/label (25) JC_ABL_modify.json')
info = json.load(f)

In [ ]:
point_data = info['_via_img_metadata']
limit = 27
for key in point_data:
    if limit <= 0:
        break
    limit-=1

    img_file_name = point_data[key]['filename']
    img_name = point_data[key]['filename'].split('.')[0]
    ori_path = [x for x in ori_img_path_list if img_name in x]
    assert len(ori_path) == 1, ori_path
    ori_img = cv2.imread(ori_path[0], cv2.IMREAD_GRAYSCALE)
    print(ori_img.shape)
    
    # get key point
    points_df = json_normalize(point_data[key]['regions'])
    if len(points_df) == 0:
        print('have not label')
        continue
    col = list(points_df.columns)
    change_name = dict(zip(col,[x.split('.')[-1] for x in col]))
    points_df.rename(columns=change_name, inplace=True)
    # print(points_df[:10])
    # set_others_value = 0.4
    key_points = []
    for id, row in points_df.iterrows():
        if row['type'] in ['CEJ', 'A_BL', 'apex']:
            key_points.append((row['cx'], row['cy']))
    mask = np.zeros_like(ori_img)
    def dist(p, q):
        return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p, q)))
    def weighted_function_1(d1, w_0=10, sigma=5):
        return w_0*math.exp(-(d1)**2/(2*sigma**2))
    for (x,y), value in np.ndenumerate(mask):
        distances = [dist((x, y), key) for key in key_points]
        mask[x][y] = weighted_function_1(min(distances))
    cv2.imwrite('./key_point_mask/'+img_file_name+'.PNG', mask)

    
